# Lab2

### 1. 
- Для кожної із адміністративних одиниць України завантажити тестові
структуровані файли, що містять значення VHI-індексу. Ця процедура має бути
автоматизована, параметром процедури має бути індекс (номер) області.
 - При зберіганні файлу до його імені потрібно додати дату та час завантаження.

- Передбачити повторні запуски скрипту, довантаження нових даних та колізію
даних;

In [1]:
import urllib.request
from datetime import datetime
import os
import glob
import pandas as pd
import re

In [2]:
directory= "./Data"

for i in range(1,29):
    url=f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2=2024&type=Mean'.format()
    
    name="NOAA_ID"+str(i)+"*"
    if glob.glob(os.path.join(directory, name)):
        print("The file already exists: " + str(i))
        continue
    
    now = datetime.now().strftime("%d%m%Y%H%M%S")
    
    wp = urllib.request.urlopen(url)
    text = wp.read()

    filename = os.path.join(directory, 'NOAA_ID'+str(i)+'_'+now+'.csv')
    out = open(filename,'wb')
    out.write(text)
 
    out.close()
    print(f"Downloaded file for {i} region")

The file already exists: 1
The file already exists: 2
The file already exists: 3
The file already exists: 4
The file already exists: 5
The file already exists: 6
The file already exists: 7
The file already exists: 8
The file already exists: 9
The file already exists: 10
The file already exists: 11
The file already exists: 12
The file already exists: 13
The file already exists: 14
The file already exists: 15
The file already exists: 16
The file already exists: 17
The file already exists: 18
The file already exists: 19
The file already exists: 20
The file already exists: 21
The file already exists: 22
The file already exists: 23
The file already exists: 24
The file already exists: 25
The file already exists: 26
The file already exists: 27
The file already exists: 28


### 2. Зчитати завантажені текстові файли у фрейм

In [3]:
def reading_text_into_the_frame(directory):
    
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    all_df = []
    fnames = os.listdir(directory)
   
    for fname in fnames:
        
        if fname.startswith("NOAA") and fname.endswith(".csv"):
            
            file_path = os.path.join(directory, fname)
            df = pd.read_csv(file_path, header=1, names=headers)
            df =df.drop(columns=['empty']).dropna()
            df = df.drop(df.loc[df['VHI'] == -1].index)
            
            i = int(re.search(r'\d+', fname).group())
            df['area'] = i
            
            df.loc[0, 'Year'] = df.loc[0,'Year'][9:]
            
            df['Year'] = df['Year'].astype(int)
            
            all_df.append(df)
            
    final_df = pd.concat(all_df, ignore_index=True)
    
    #final_df = final_df.drop(columns=['empty'])
    print(final_df)
    
    print(final_df.dtypes)
    
    return(final_df)

final_df = reading_text_into_the_frame(directory)

       Year  Week    SMN     SMT    VCI    TCI    VHI  area
0      1982   1.0  0.059  258.24  51.11  48.78  49.95    10
1      1982   2.0  0.063  261.53  55.89  38.20  47.04    10
2      1982   3.0  0.063  263.45  57.30  32.69  44.99    10
3      1982   4.0  0.061  265.10  53.96  28.62  41.29    10
4      1982   5.0  0.058  266.42  46.87  28.57  37.72    10
...     ...   ...    ...     ...    ...    ...    ...   ...
61035  2024  42.0  0.152  291.74  40.24   9.54  24.92     9
61036  2024  43.0  0.150  289.53  43.73   6.85  25.30     9
61037  2024  44.0  0.148  287.22  46.47   2.35  24.41     9
61038  2024  45.0  0.148  285.25  50.19   0.76  25.48     9
61039  2024  46.0  0.145  283.59  52.70   0.39  26.55     9

[61040 rows x 8 columns]
Year      int64
Week    float64
SMN     float64
SMT     float64
VCI     float64
TCI     float64
VHI     float64
area      int64
dtype: object


### 3. Реалізувати окрему процедуру, яка змінить індекси областей, за англійською абеткою на наступні, за українською:

In [4]:
def change_index():
    
    
    change = {1:22, 2:24, 3:23, 4:25, 5:3, 6:4, 7:8, 8:19, 9:20, 10:21, 
                11:9, 12:9, 13:10,  14:11, 15:12, 16:13, 17:14, 18:15 ,19:16, 
                20:25, 21:17 ,22:18 ,23:6 ,24:1 ,25:2 ,26:7 ,27:5}
    
    final_df["area"] = final_df["area"].replace(change)
    
    print(final_df)


change_index()



       Year  Week    SMN     SMT    VCI    TCI    VHI  area
0      1982   1.0  0.059  258.24  51.11  48.78  49.95    21
1      1982   2.0  0.063  261.53  55.89  38.20  47.04    21
2      1982   3.0  0.063  263.45  57.30  32.69  44.99    21
3      1982   4.0  0.061  265.10  53.96  28.62  41.29    21
4      1982   5.0  0.058  266.42  46.87  28.57  37.72    21
...     ...   ...    ...     ...    ...    ...    ...   ...
61035  2024  42.0  0.152  291.74  40.24   9.54  24.92    20
61036  2024  43.0  0.150  289.53  43.73   6.85  25.30    20
61037  2024  44.0  0.148  287.22  46.47   2.35  24.41    20
61038  2024  45.0  0.148  285.25  50.19   0.76  25.48    20
61039  2024  46.0  0.145  283.59  52.70   0.39  26.55    20

[61040 rows x 8 columns]


### 4.1 Реалізувати ряд VHI для області за вказаний рік;
    

In [5]:
def vhi_region(index, year):
    
    vhi = final_df[(final_df['area'] == index) & (final_df["Year"] == year)][['Year','area','VHI']]
    return vhi

print(vhi_region(4, 1995))

       Year  area    VHI
52969  1995     4  62.15
52970  1995     4  53.02
52971  1995     4  41.55
52972  1995     4  31.77
52973  1995     4  26.08
52974  1995     4  23.32
52975  1995     4  22.68
52976  1995     4  24.90
52977  1995     4  27.88
52978  1995     4  31.57
52979  1995     4  32.75
52980  1995     4  34.11
52981  1995     4  36.30
52982  1995     4  41.11
52983  1995     4  46.10
52984  1995     4  48.52
52985  1995     4  49.62
52986  1995     4  49.75
52987  1995     4  48.84
52988  1995     4  46.96
52989  1995     4  48.42
52990  1995     4  53.67
52991  1995     4  57.77
52992  1995     4  59.65
52993  1995     4  60.47
52994  1995     4  60.22
52995  1995     4  59.26
52996  1995     4  58.27
52997  1995     4  57.26
52998  1995     4  55.51
52999  1995     4  53.81
53000  1995     4  52.98
53001  1995     4  51.30
53002  1995     4  50.40
53003  1995     4  53.62
53004  1995     4  56.03
53005  1995     4  56.42
53006  1995     4  54.59
53007  1995     4  52.79


### 4.2 Реалізувати пошук екстремумів (min та max) для вказаних областей та років, середнього та медіани;

In [6]:
def extremes(year, regions):
    
    final_df['Year'] = pd.to_numeric(final_df['Year'], errors='coerce')
    final_df['area'] = pd.to_numeric(final_df['area'], errors='coerce')
    
    data = final_df[(final_df['Year'] >= year[0]) &
                    (final_df['Year'] <= year[1]) &
                    (final_df['area'] >= regions[0]) & 
                    (final_df['area'] <= regions[1])]
    
    min_vhi = data['VHI'].min()
    max_vhi = data['VHI'].max()
    mean_vhi = data['VHI'].mean()
    median_vhi = data['VHI'].median()
    
    return("Extremes: \n   min-" + str(min_vhi) +"\n   max-" + str(max_vhi) +"\n   mean-" + str(mean_vhi) +"\n   min-" + str(median_vhi))

print(extremes([ 1984, 1992], [3, 11]))
    
    

Extremes: 
   min-12.45
   max-76.87
   mean-43.680641137855574
   min-43.2


### 4.3 Реалізувати ряд VHI за вказаний діапазон років для вказаних областей;

In [7]:
def sf_range(year, region):
    
    data = final_df[(final_df['Year'].between(year[0],year[1]))&
                    (final_df['area'].isin(region))]
    
    f_data = data.sort_values(by='area')
    
    return f_data[['Year', 'area', 'VHI']]

print(sf_range([1987, 1994], [4, 11]))

       Year  area    VHI
52968  1994     4  28.74
52967  1994     4  31.57
52966  1994     4  36.93
52965  1994     4  42.42
52964  1994     4  47.86
...     ...   ...    ...
8989   1987    11  42.26
8988   1987    11  45.26
8987   1987    11  47.96
8986   1987    11  49.05
8985   1987    11  49.84

[800 rows x 3 columns]


### 4.4 Виявити роки, протягом яких екстремальні посухи торкнулися більше вказаного відсотка областей по Україні 


In [8]:
def droughts(percent):
    
    count = int((percent*25)/100)
    print(count)
    drought = final_df[final_df['VHI']<15]
    count_of_region = drought.groupby(by='Year')['area'].count()
    years_d = count_of_region[count_of_region > count].index
    
    drought_avg = drought[drought['Year'].isin(years_d)].groupby(['Year', 'area' ],as_index=False)['VHI'].mean()
    
    #return drought[drought['Year'].isin(years_d)][['Year', 'VHI', 'area']]
    return drought_avg[['Year', 'VHI', 'area']]

print(droughts(20))
    

5
    Year        VHI  area
0   1993   8.872000     4
1   1993  14.080000    11
2   2000  12.213333     1
3   2000  10.436667     9
4   2000  11.460000    19
5   2000  12.415000    22
6   2000  10.828571    25
7   2007  12.641818     7
8   2007   7.895000    13
9   2007   8.114444    14
10  2007  13.289091    20
11  2007  14.008333    25
